In [ ]:
!pip install langchain langchain-ollama ollama
!pip install pypdf faiss-cpu pytest boto3
!pip install langchain-community


In [1]:
import ollama

In [10]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings


def get_embedding_function():
    # embeddings = BedrockEmbeddings(
    #     credentials_profile_name="default", region_name="us-east-1"
    # )
    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    return embeddings

In [2]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma


CHROMA_PATH = "chroma"
DATA_PATH = "data"


def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()


def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


def add_to_chroma(chunks: list[Document]):
    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function()
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)



In [3]:
documents = load_documents()
print(documents)
#chunks = split_documents(documents)
#add_to_chroma(chunks)

[Document(metadata={'source': 'data\\31_2021_ND-CP_462291.pdf', 'page': 0}, page_content='CHÍNH PHỦ  \n---------  CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM  \nĐộc lập - Tự do - Hạnh phúc  \n---------------  \nSố: 31/2021/NĐ -CP Hà Nội, ngày 26 tháng 3 năm 2021  \n  \nNGHỊ ĐỊNH  \nQUY ĐỊNH CHI TIẾT VÀ HƯỚNG DẪN THI HÀNH MỘT SỐ ĐIỀU CỦA LUẬT ĐẦU TƯ  \nCăn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015; Luật sửa đổi, bổ sung một số điều \ncủa Luật Tổ chức Chính phủ và Luật Tổ chức chính quyền địa phương ngày 22 tháng 11 năm \n2019;  \nCăn cứ Luật Đầu tư ngày 17 tháng 6 năm 2020;  \nCăn cứ Luật Doanh nghiệp ngày 17 tháng 6 năm 2020;  \nTheo đề nghị của Bộ trưởng Bộ Kế hoạch và Đầu tư;  \nChính phủ ban hành Nghị định quy định chi tiết và hướng dẫn thi hành một số điều của Luật \nĐầu tư.  \nChương I  \nQUY ĐỊNH CHUNG  \nĐiều 1. Phạm vi điều chỉnh và đối tượng áp dụng  \n1. Nghị định này quy định chi tiết và hướng dẫn thi hành một số điều của Luật Đầu tư về điều \nkiện đầu tư kinh doanh; ngành ,

In [4]:
chunks = split_documents(documents)

In [5]:
chunks_with_id =  calculate_chunk_ids(chunks)

In [7]:
print(chunks_with_id[0])

page_content='CHÍNH PHỦ  
---------  CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM  
Độc lập - Tự do - Hạnh phúc  
---------------  
Số: 31/2021/NĐ -CP Hà Nội, ngày 26 tháng 3 năm 2021  
  
NGHỊ ĐỊNH  
QUY ĐỊNH CHI TIẾT VÀ HƯỚNG DẪN THI HÀNH MỘT SỐ ĐIỀU CỦA LUẬT ĐẦU TƯ  
Căn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015; Luật sửa đổi, bổ sung một số điều 
của Luật Tổ chức Chính phủ và Luật Tổ chức chính quyền địa phương ngày 22 tháng 11 năm 
2019;  
Căn cứ Luật Đầu tư ngày 17 tháng 6 năm 2020;  
Căn cứ Luật Doanh nghiệp ngày 17 tháng 6 năm 2020;  
Theo đề nghị của Bộ trưởng Bộ Kế hoạch và Đầu tư;  
Chính phủ ban hành Nghị định quy định chi tiết và hướng dẫn thi hành một số điều của Luật 
Đầu tư.  
Chương I  
QUY ĐỊNH CHUNG  
Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng' metadata={'source': 'data\\31_2021_ND-CP_462291.pdf', 'page': 0, 'id': 'data\\31_2021_ND-CP_462291.pdf:0:0'}


In [8]:
!pip install faiss-cpu


     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.6 MB 1.3 MB/s eta 0:00:12
     ---------------------------------------- 0.1/14.6 MB 1.1 MB/s eta 0:00:14
     ---------------------------------------- 0.2/14.6 MB 1.5 MB/s eta 0:00:10
      --------------------------------------- 0.3/14.6 MB 2.0 MB/s eta 0:00:08
     - -------------------------------------- 0.5/14.6 MB 2.3 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.5 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/14.6 MB 2.7 MB/s eta 0:00:06
     -- ------------------------------------- 0.9/14.6 MB 2.7 MB/s eta 0:00:05
     -- ------------------------------------- 1.1/14.6 MB 2.8 MB/s eta 0:00:05
     --- ------------------------------------ 1.2/14.6 MB 2.9 MB/s eta 0:00:05
     --- ------------------------------------ 1.4/14.6 MB 3.1 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/14.6 MB 3


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
collections = client.list_collections()

In [11]:
embeddings = get_embedding_function()

In [12]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("pdf")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [14]:
ids = []
documents = []
for chunk in chunks_with_id:
  id =  chunk.metadata.get('id')
  ids.append(id)
  document =  chunk.page_content
  documents.append(document)
vector_store.add_documents(
    documents=chunks_with_id
  )

['2d37b3ab-3cd4-4b5f-af78-08b3138e6311',
 '6160f7cf-ead4-4d35-81cf-090dc96c13e5',
 'fc9e0fcf-5835-4ed6-8be2-a0336d7e29f7',
 '1e2e51d2-c512-4eca-bc89-0c255745e21e',
 '5a86f7f2-2c68-4ac4-aeb7-bb6b175a0bd7',
 'f655052c-f0c3-44cc-a05d-bc3cc478325b',
 '72a2d02f-367c-4d45-81c8-3eaebb83cefc',
 'ac244712-cc56-4408-8d25-1455e2142fa7',
 '411d265f-0fc5-44bf-be0f-06ed2e064348',
 'e621f5b2-ae48-4623-948c-2d52d18656be',
 '8959156b-dc5a-430f-8bc4-20be71eef8db',
 '29538f00-ff09-4ea5-b321-f141dc370801',
 'd87753d1-e035-4169-8ea7-853eff2fbcc8',
 'c4901599-71af-4961-9a2a-a89b1259bf7c',
 '7a4c481a-25c9-40a2-886d-75b82ab80eeb',
 'e2af84ae-b05d-46f2-8ede-ad4ab867fb0f',
 'bc57ec09-fa43-4919-8e98-2e04b51aeddd',
 'a107efb4-6fcc-422b-9c27-536ef667368d',
 'dcee7974-bb55-4748-acb6-90270d07303e',
 '7808704f-b6e5-4717-a88d-db62bb2488c5',
 'ff4ee481-0a4c-42a5-87ac-f4fc87d2d4bf',
 'fc530cec-25b1-43d4-b5f0-5aa4c89299f6',
 '300b5437-5a0f-4007-a868-7d0fcc8d4953',
 '25401a0f-3f30-477f-8a52-6e38d0ebaf4c',
 '7f2c0f93-e373-

In [23]:
import argparse
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
PROMPT_TEMPLATE = """
Trả lời câu hỏi dựa theo thông tin ngữ cảnh được cung cấp dưới đây:

{context}

---

Hãy trả lời bằng tiếng Việt
Trả lời câu hỏi sau đây dựa theo thông tin ngữ cảnh ở trên: {question}
"""
def query_rag(query_text: str):
    # Prepare the DB.
    embedding_function = get_embedding_function()
    #db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = vector_store.similarity_search_with_score(query_text, k=5)
    # for res, score in results:
    #     print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")    
    # print('\n--------------------------------------\n')
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    print(context_text)
    print('\n--------------------------------------\n')
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = OllamaLLM(model="llama3")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text


In [24]:
print("Xin chào tôi là AI chatbot. Hãy gõ exit để thoát")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    print("Bot: ",query_rag(user_input))

Xin chào tôi là AI chatbot. Hãy gõ exit để thoát
* [SIM=340.495728] sự điều chỉnh khác so với khung giá và các loại phí sử dụng hạ tầng đã đăng ký;  
c) Thu các loại phí sử dụng hạ tầng; [{'source': 'data\\31_2021_ND-CP_462291.pdf', 'page': 52, 'id': 'data\\31_2021_ND-CP_462291.pdf:52:4'}]
* [SIM=343.268188] doanh nghiệp hoặc pháp luật khác tương ứng với từng loại hình tổ chức kinh tế. [{'source': 'data\\31_2021_ND-CP_462291.pdf', 'page': 53, 'id': 'data\\31_2021_ND-CP_462291.pdf:53:4'}]
* [SIM=361.998047] lực.  
3. Đối với dự án đầu tư ho ạt động theo Gi ấy chứng nh ận đầu tư (đ ồng th ời là Gi ấy chứng nh ận đăng ký 
kinh doanh) ho ặc Giấy phép đ ầu tư, Cơ quan đăng ký đ ầu tư quy ết định ch ấm dứt hoạt động của dự án 
đầu tư mà không thu h ồi Giấy chứng nh ận đầu tư (đ ồng th ời là Gi ấy chứng nh ận đăng ký kinh doanh) 
hoặc Giấy phép đ ầu tư. Trong trư ờng hợp này, n ội dung đăng ký kinh doanh t ại Giấy chứng nh ận đầu tư 
(đồng th ời là Gi ấy chứng nh ận đăng ký kinh doanh), Gi ấy